In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:

import numpy as np
import pandas as pd
import pickle
import numpy as np
import random
import time
import os
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
dataset_dir = "drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/"
work_dir = "drive/My Drive/HAM10000/"

In [4]:
list_categories = ['akiec','bcc','bkl','df','mel','nv','vasc']
num_classes = len(list_categories)
print('num_classes =', num_classes)

num_classes = 7


In [5]:
# Saving files:
'''
np.save(work_dir+"np_test_imageset.pkl", np_test_imageset, allow_pickle=True)
np.save(work_dir+"np_val_imageset.pkl", np_val_imageset, allow_pickle=True)
np.save(work_dir+"np_train_imageset.pkl", np_train_imageset, allow_pickle=True)

np.save(work_dir+"np_test_label.pkl", np_test_label, allow_pickle=True)
np.save(work_dir+"np_val_label.pkl", np_val_label, allow_pickle=True)
np.save(work_dir+"np_train_label.pkl", np_train_label, allow_pickle=True)
#'''

'\nnp.save(work_dir+"np_test_imageset.pkl", np_test_imageset, allow_pickle=True)\nnp.save(work_dir+"np_val_imageset.pkl", np_val_imageset, allow_pickle=True)\nnp.save(work_dir+"np_train_imageset.pkl", np_train_imageset, allow_pickle=True)\n\nnp.save(work_dir+"np_test_label.pkl", np_test_label, allow_pickle=True)\nnp.save(work_dir+"np_val_label.pkl", np_val_label, allow_pickle=True)\nnp.save(work_dir+"np_train_label.pkl", np_train_label, allow_pickle=True)\n#'

In [6]:
# Loading files:
np_test_imageset = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_norm_test_X.pkl.npy", allow_pickle=True)
np_val_imageset = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_norm_val_X.pkl.npy", allow_pickle=True)
np_train_imageset = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_norm_train_X.pkl.npy", allow_pickle=True)

np_test_label = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_test_Y.pkl.npy", allow_pickle=True)
np_val_label = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_val_Y.pkl.npy", allow_pickle=True)
np_train_label = np.load(dataset_dir+"HAM10000_Gray_Categorized_train_Upsampled_128x128_train_Y.pkl.npy", allow_pickle=True)

In [7]:
final_test_imageset = np.expand_dims(np_test_imageset, axis = 3)
final_val_imageset = np.expand_dims(np_val_imageset, axis = 3)
final_train_imageset = np.expand_dims(np_train_imageset, axis = 3)

test_label2 = np.expand_dims(np_test_label, axis = 1)
val_label2 = np.expand_dims(np_val_label, axis = 1)
train_label2 = np.expand_dims(np_train_label, axis = 1)

print('final_test_imageset.shape =', final_test_imageset.shape)
print('final_val_imageset.shape =', final_val_imageset.shape)
print('final_train_imageset.shape =', final_train_imageset.shape)

print('\ntest_label2.shape =', test_label2.shape)
print('val_label2.shape =', val_label2.shape)
print('train_label2.shape =', train_label2.shape)

final_test_imageset.shape = (998, 128, 128, 1)
final_val_imageset.shape = (998, 128, 128, 1)
final_train_imageset.shape = (37555, 128, 128, 1)

test_label2.shape = (998, 1)
val_label2.shape = (998, 1)
train_label2.shape = (37555, 1)


In [8]:
final_test_label = tf.keras.utils.to_categorical(test_label2, num_classes)
final_val_label = tf.keras.utils.to_categorical(val_label2, num_classes)
final_train_label = tf.keras.utils.to_categorical(train_label2, num_classes)

print('final_test_label.shape =',final_test_label.shape)
print('final_val_label.shape =',final_val_label.shape)
print('final_train_label.shape =',final_train_label.shape)

final_test_label.shape = (998, 7)
final_val_label.shape = (998, 7)
final_train_label.shape = (37555, 7)


In [9]:
input_shape = final_train_imageset.shape[1:]

# Input tensor shape
inputs = Input(input_shape)
x = MaxPooling2D(pool_size=(3,3), strides=(1,1))

x = Conv2D(255, kernel_size=(3,3), activation='relu')(inputs)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(255, kernel_size=(3,3), activation='relu')(x)

x = Flatten()(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 255)     2550      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 255)     585480    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 122, 255)     585480    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 120, 255)     585480    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 118, 118, 255)     585480    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 116, 116, 255)    

In [10]:
'''
#adam = tf.keras.optimizers.Adam(lr = 0.01) #, beta_1=0.9, beta_2=0.999, amsgrad=False
optimizer = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False) #, beta_1=0.9, beta_2=0.999, amsgrad=False
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])
#'''

"\n#adam = tf.keras.optimizers.Adam(lr = 0.01) #, beta_1=0.9, beta_2=0.999, amsgrad=False\noptimizer = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False) #, beta_1=0.9, beta_2=0.999, amsgrad=False\n#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nmodel.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])\n#"

In [11]:
optimizer = tf.keras.optimizers.RMSprop(lr = 0.0001, epsilon=1e-4)
model.compile(
    optimizer=optimizer, #'rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# save the best model with least validation loss
checkpointer_name  = "weights_HAM10000_SimpleNet2.hdf5"
checkpointer = ModelCheckpoint(filepath = "drive/My Drive/HAM10000/"+checkpointer_name, verbose = 1, save_best_only = True)

In [13]:
start_time = time.time()
history = model.fit(final_train_imageset, final_train_label, 
                    shuffle=False, 
                    batch_size = 1, 
                    epochs = 3, 
                    #steps_per_epoch = 2,
                    #validation_split = 0.1, 
                    validation_data = (final_val_imageset,final_val_label),
                    callbacks=[checkpointer]
                    )
elapsed_time = time.time() - start_time 
print("\nTime elapsed: ", elapsed_time)

Epoch 1/3
37555/37555 [==============================] - ETA: 0s - loss: 1.9460 - accuracy: 0.0916WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_test_batch_end` time: 0.0125s). Check your callbacks.

Epoch 00001: val_loss improved from inf to 1.94592, saving model to drive/My Drive/HAM10000/weights_HAM10000_SimpleNet2.hdf5
37555/37555 [==============================] - 2334s 62ms/step - loss: 1.9460 - accuracy: 0.0916 - val_loss: 1.9459 - val_accuracy: 0.0140
Epoch 2/3
 2366/37555 [>.............................] - ETA: 36:05 - loss: 1.9460 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
result = model.evaluate(final_test_imageset, final_test_label)
print("Accuracy : {}, Loss: {}".format(result[1], result[0]))

In [ ]:
model_loaded = load_model(work_dir+checkpointer_name)

In [ ]:
result2 = model_loaded.evaluate(final_test_imageset, final_test_label)
print("Accuracy : {}, Loss: {}".format(result2[1], result2[0]))

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model_loaded.predict_generator(final_test_imageset, len(final_test_imageset))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(np_test_label, y_pred))

In [ ]:
'''
print('Actual:', np_test_label)
print('Predic:', y_pred)
#'''

In [ ]:
# Precision [TP/TP+FP] = The ratio of correctly predicted positive observations to the total predicted positive observations.
# Recall (Sensitivity) [TP/TP+FN] = The ratio of correctly predicted positive observations to the all observations in actual class - 'yes'.
# F1 score [F1 Score = 2*(Recall * Precision) / (Recall + Precision)] = The weighted average of Precision and Recall.
# Support = The number of samples of the true response that lie in that class.
print('Classification Report:')
#target_names = ['Mono', 'Di'] # not ['Di', 'Mono']
print(classification_report(np_test_label, y_pred)) #, target_names=target_names))

In [ ]:
print('Classification Report')
print(classification_report(np_test_label, y_pred, target_names=list_categories))

In [ ]:
# Getting the model history keys 
history.history.keys()

In [ ]:
# plot the training artifacts

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'best')
plt.show()